In [ ]:
# coding: utf-8

# In[19]:


from tkinter import *
import sqlite3
import tkinter as tk
import cv2
import PIL.Image
import PIL.ImageTk
import time
from tkinter import messagebox

import result as hand
import result_head as helmet
# In[20]:


class App:
    def __init__(self, window, window_title, video_source=0):
        self.window = window
        self.window.title(window_title)
        self.video_source = video_source
        self.vid = MyVideoCapture(self.video_source)
        self.canvas = tk.Canvas(window, width = self.vid.width, height = self.vid.height)
        self.canvas.pack()
        self.btn_snapshot=tk.Button(window, text="Start Scanning", width=0, command=self.snapshot)
        self.btn_snapshot.pack(anchor=tk.CENTER, expand=True)
        self.delay = 15
        self.update()
        self.window.mainloop()
    def snapshot(self):
        tk.messagebox.showinfo("Are You Ready", "Scanninig Head Part")
        ret, frame = self.vid.get_frame()
        if ret :
            img="Helmet" + ".jpg"
            cv2.imwrite(img, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
        self.delay = 15
        self.update()
        tk.messagebox.showinfo("Are You Ready", "Scanning Hands Part ")
        ret, frame = self.vid.get_frame()
        if ret :
            cv2.imwrite("Gloves" + ".jpg", cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
        self.delay = 15
        self.update()
        tk.messagebox.showinfo("Are You Ready", "Scanning Legs Part ")
        ret, frame = self.vid.get_frame()
        if ret :
            cv2.imwrite("Shoes" + ".jpg", cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    def update(self):
        ret, frame = self.vid.get_frame()
        if ret:
            self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(frame))
            self.canvas.create_image(0, 0, image = self.photo, anchor = tk.NW)
            self.window.after(self.delay, self.update)


# In[21]:


class MyVideoCapture:
    def __init__(self, video_source=0):
        self.vid = cv2.VideoCapture(video_source)
        if not self.vid.isOpened():
             raise ValueError("Unable to open video source", video_source)
        self.width = self.vid.get(cv2.CAP_PROP_FRAME_WIDTH)
        self.height = self.vid.get(cv2.CAP_PROP_FRAME_HEIGHT)
    def get_frame(self):
        if self.vid.isOpened():
            ret, frame = self.vid.read()
            if ret:
                return (ret, cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            else:
                return (ret, None)
        else:
            return (ret, None)
    def __del__(self):
        if self.vid.isOpened():
            self.vid.release()


# In[22]:


def capture(root):
    root.iconify()
    App(tk.Toplevel(), "Live Image Capture")


# In[23]:


def check():
    LempID=empID.get()
    conn = sqlite3.connect('OperatorSafetyGear.db')
    with conn:
        cursor=conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS Employee_Attendance(EmployeeID TEXT,Date TEXT,Status TEXT)')
    if hand.predict_category("Gloves.jpg")==0 and helmet.predict_category("Helmet.jpg")==1:
        cursor.execute('INSERT INTO Employee_Attendance(EmployeeID,Date,Status) SELECT Employee_details.EmployeeID,CURRENT_TIMESTAMP,"Present" FROM Employee_details WHERE EXISTS(SELECT Employee_details.EmployeeID FROM Employee_details WHERE Employee_details.EmployeeID=?)',(LempID,))
        cursor.execute('SELECT * FROM Employee_Attendance')
        conn.commit()
        tk.messagebox.showinfo("Status", "Access Granted")
    else:
        cursor.execute('INSERT INTO Employee_Attendance(EmployeeID,Date,Status) SELECT Employee_details.EmployeeID,CURRENT_TIMESTAMP,"Absent" FROM Employee_details WHERE EXISTS(SELECT Employee_details.EmployeeID FROM Employee_details WHERE Employee_details.EmployeeID=?)',(LempID,))
        cursor.execute('SELECT * FROM Employee_Attendance')
        conn.commit()
        tk.messagebox.showinfo("Status", "Access Denied")

# In[24]:


root = tk.Tk()
root.resizable(0, 0)
root.title("Attendance")
root.geometry('500x500')
empID=StringVar()
date=StringVar()
label_0 = Label(root, text="Attendance",width=20,font=("bold", 20))
label_0.place(x=90,y=53)
label_1 = Label(root, text="Employee ID",width=20,font=("bold", 10))
label_1.place(x=80,y=180)
entry_1 = Entry(root,textvar=empID)
entry_1.place(x=240,y=180)
Button(root, text='Capture',width=20,bg='brown',fg='white',command=lambda:capture(root)).place(x=180,y=280)
Button(root, text='Submit',width=20,bg='brown',fg='white',command=check).place(x=180,y=380)
root.mainloop()
